In [11]:
# Install first: pip install undetected-chromedriver selenium beautifulsoup4 pandas
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
jumia = requests.get('https://www.jumia.co.ke/all-products/')

In [ ]:
jumia.content

In [13]:
# --- Launch Chrome with undetected-chromedriver (visible window) ---
# Remove headless for maximum compatibility
driver = uc.Chrome(headless=False)  # shows real browser window


In [14]:
# --- Helper to scroll down for lazy-loading ---
def scroll_page(driver, scroll_pause=3.0, max_scrolls=3):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


In [15]:
# --- Scrape Jumia all‑products pages ---
name_info, price_info, rating_info, link_info = [], [], [], []

for page in range(1, 51):
    url = f"https://www.jumia.co.ke/all-products/?page={page}#catalog-listing"
    driver.get(url)
    time.sleep(5)  # let JS load

    scroll_page(driver, scroll_pause=3.0, max_scrolls=2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.select("article.prd")  # product cards

    for p in products:
        name_tag = p.select_one("h3.name")
        price_tag = p.select_one("div.prc")
        rating_tag = p.select_one("div.stars") or p.select_one("div.rev")
        link_tag = p.select_one("a.core")

        name_info.append(name_tag.get_text(strip=True) if name_tag else "N/A")
        price_info.append(price_tag.get_text(strip=True) if price_tag else "N/A")
        rating_info.append(rating_tag.get_text(strip=True) if rating_tag else "N/A")
        link_info.append("https://www.jumia.co.ke" + link_tag["href"] if link_tag else "N/A")

    print(f"Scraped page {page} with {len(products)} products")


Scraped page 1 with 40 products
Scraped page 2 with 40 products
Scraped page 3 with 40 products
Scraped page 4 with 40 products
Scraped page 5 with 40 products
Scraped page 6 with 40 products
Scraped page 7 with 40 products
Scraped page 8 with 40 products
Scraped page 9 with 40 products
Scraped page 10 with 40 products
Scraped page 11 with 40 products
Scraped page 12 with 40 products
Scraped page 13 with 40 products
Scraped page 14 with 40 products
Scraped page 15 with 40 products
Scraped page 16 with 40 products
Scraped page 17 with 40 products
Scraped page 18 with 40 products
Scraped page 19 with 40 products
Scraped page 20 with 40 products
Scraped page 21 with 40 products
Scraped page 22 with 40 products
Scraped page 23 with 40 products
Scraped page 24 with 40 products
Scraped page 25 with 40 products
Scraped page 26 with 40 products
Scraped page 27 with 40 products
Scraped page 28 with 40 products
Scraped page 29 with 40 products
Scraped page 30 with 40 products
Scraped page 31 wit

In [19]:
# --- Close driver ---
driver.quit()

In [25]:
# --- Build DataFrame & save to CSV ---
df = pd.DataFrame({
    "Product Name": name_info,
    "Price": price_info,
    "Rating": rating_info
})

# csv_file = "jumia_products.csv"
# df.to_csv(csv_file, index=False)
# print(f"Total rows: {len(df)} — saved to {csv_file}")
df.head()


,Product Name,Price,Rating
0,"Sonar Stainless Steel 1.8Ltr Electric Kettle,1...",KSh 523,4 out of 5
1,6Pcs Soft Cotton Checked Men's Boxers – Multic...,KSh 586,3.6 out of 5
2,"Starlink Mini kit : High-Speed, Low-Latency In...","KSh 24,000",4.5 out of 5
3,"Sonar 2L Cordless Electric Kettle ,1500W Energ...",KSh 599,4.1 out of 5
4,"VILLAON V101 1.77"" Kabambe, Wireless FM, Camer...",KSh 850,3.8 out of 5
